### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
# Calculate the total number of schools
total_schools = school_data_complete['school_name'].nunique()

# Calculate the total number of students
total_students = school_data_complete['Student ID'].count()

# Calculate the total budget
total_budget = school_data_complete['budget'].unique().sum()

# Calculate the average math score
ave_math_score = school_data_complete['math_score'].mean()

# Calculate the average reading score
ave_reading_score = school_data_complete['reading_score'].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
passing_math_df = school_data_complete.loc[school_data_complete['math_score'] >= 70]
prcnt_passing_math = passing_math_df['Student ID'].count()*100/total_students

# Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading_df = school_data_complete.loc[school_data_complete['reading_score'] >= 70]
prcnt_passing_reading = passing_reading_df['Student ID'].count()*100/total_students

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
#school_data_complete['ave_score'] = (school_data_complete['math_score'] + school_data_complete['reading_score'])/2
#passing_df = school_data_complete.loc[school_data_complete['ave_score'] >= 70]
#prcnt_passing = passing_df['Student ID'].count()*100/total_students


# Create a dataframe to hold the above results
districts = ({
    'Total Schools' : [total_schools], 
    'Total Students' : [total_students], 
    'Total Budget' : [total_budget], 
    'Average Math Score' : [ave_math_score], 
    'Average Reading Score' : [ave_reading_score], 
    '% Passing Math' : [prcnt_passing_math], 
    '% Passing Reading' : [prcnt_passing_reading] 
#    '% Overall Passing Rate' : [prcnt_passing]
})
    
district_summary = pd.DataFrame(districts)

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
district_summary['% Overall Passing Rate'] = (district_summary['Average Math Score'] + district_summary['Average Reading Score'])/2 

# Data formatting
district_summary['Total Students'] = district_summary['Total Students'].map('{:,}'.format)
district_summary['Total Budget'] = district_summary['Total Budget'].map('${:,}'.format)
district_summary['Average Math Score'] = district_summary['Average Math Score'].map('{:.2f}'.format)
district_summary['Average Reading Score'] = district_summary['Average Reading Score'].map('{:.2f}'.format)
district_summary['% Passing Math'] = district_summary['% Passing Math'].map('{:.2f}%'.format)
district_summary['% Passing Reading'] = district_summary['% Passing Reading'].map('{:.2f}%'.format)
district_summary['% Overall Passing Rate'] = district_summary['% Overall Passing Rate'].map('{:.2f}%'.format)

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,80.43%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [4]:
# Scores of all students
all_students = pd.DataFrame({
        'School Name': school_data_complete['school_name'],
        'School Type': school_data_complete['type'],
        'Total Students': school_data_complete['size'],
        'Total School Budget': school_data_complete['budget'],
        'Per Student Budget': school_data_complete['budget']/school_data_complete['size'],
        'Average Math Score': school_data_complete['math_score'],
        'Average Reading Score': school_data_complete['reading_score'],
        'Average Overall Score': (school_data_complete['math_score'] + school_data_complete['reading_score'])/2
})

# Average scores grouped by school
school_group = all_students.groupby(['School Name']).mean()

# Add back School Type
school_type = all_students.loc[:,['School Name','School Type']].drop_duplicates()
school_type.set_index('School Name', inplace=True)
school_group_type = pd.merge(school_type, school_group, on='School Name')

# % Passing Math
passing_math_students = all_students.loc[all_students['Average Math Score'] >= 70]
passing_math_count = passing_math_students['School Name'].value_counts().sort_index()
school_group_type['% Passing Math'] = passing_math_count*100/school_group_type['Total Students']

# % Passing Reading
passing_reading_students = all_students.loc[all_students['Average Reading Score'] >= 70]
passing_reading_count = passing_reading_students['School Name'].value_counts().sort_index()
school_group_type['% Passing Reading'] = passing_reading_count*100/school_group_type['Total Students']

# % Overall Passing Rate
# passing_students = all_students.loc[all_students['Average Overall Score'] >= 70]
# passing_students = all_students.loc[(all_students['Average Math Score'] >= 70) & (all_students['Average Reading Score'] >= 70)]
# passing_count = passing_students['School Name'].value_counts().sort_index()
# school_group_type['% Overall Passing Rate'] = passing_count*100/school_group_type['Total Students']

# % Overall Passing Rate
school_group_type['% Overall Passing Rate'] = (school_group_type['% Passing Math'] + school_group_type['% Passing Reading'])/2

# Data Formatting
school_group_type['Total Students'] = school_group_type['Total Students'].map('{:,.0f}'.format)
school_group_type['Total School Budget'] = school_group_type['Total School Budget'].map('${:,.2f}'.format)
school_group_type['Per Student Budget'] = school_group_type['Per Student Budget'].map('${:.2f}'.format)
school_group_type['Average Math Score'] = school_group_type['Average Math Score'].map('{:.2f}'.format)
school_group_type['Average Reading Score'] = school_group_type['Average Reading Score'].map('{:.2f}'.format)
school_group_type['Average Overall Score'] = school_group_type['Average Overall Score'].map('{:.2f}'.format)
school_group_type['% Passing Math'] = school_group_type['% Passing Math'].map('{:.2f}%'.format)
school_group_type['% Passing Reading'] = school_group_type['% Passing Reading'].map('{:.2f}%'.format)
school_group_type['% Overall Passing Rate'] = school_group_type['% Overall Passing Rate'].map('{:.2f}%'.format)

school_group_type.sort_index()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Average Overall Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,79.04,66.68%,81.93%,74.31%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,83.52,94.13%,97.04%,95.59%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,78.93,65.99%,80.74%,73.36%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,78.92,68.31%,79.30%,73.80%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,83.58,93.39%,97.14%,95.27%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,79.11,66.75%,80.86%,73.81%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,83.81,92.51%,96.25%,94.38%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,78.91,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,79.02,66.06%,81.22%,73.64%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [5]:
# Top performing schools
top_schools = school_group_type.sort_values('% Overall Passing Rate', ascending=False).reset_index()
top_schools.set_index('School Name', inplace=True)
top_schools.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Average Overall Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,83.52,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,83.63,93.27%,97.31%,95.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,83.58,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,83.94,94.59%,95.95%,95.27%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,83.63,93.87%,96.54%,95.20%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [6]:
# Bottom performing schools
bottom_schools = school_group_type.sort_values('% Overall Passing Rate').reset_index()
bottom_schools.set_index('School Name', inplace=True)
bottom_schools.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Average Overall Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,78.79,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,78.93,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,78.91,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,79.02,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,78.92,68.31%,79.30%,73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [5]:
# New dataframe to hold math scores
math_scores_complete = pd.DataFrame({
    'School Name': school_data_complete['school_name'],
    'Grade': school_data_complete['grade'],
    'Math Score': school_data_complete['math_score']
})

# Math scores grouped by school and grade
math_scores = math_scores_complete.groupby(['School Name','Grade']).mean()

# Pivot table to put grades as column headers
math_scores_pivot = math_scores.pivot_table(index='School Name', columns='Grade', values='Math Score')
math_scores_pivot = math_scores_pivot[['9th','10th','11th','12th']]
math_scores_pivot

Grade,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [6]:
# New dataframe to hold reading scores
reading_scores_complete = pd.DataFrame({
    'School Name': school_data_complete['school_name'],
    'Grade': school_data_complete['grade'],
    'Reading Score': school_data_complete['reading_score']
})

# Reading scores grouped by school and grade
reading_scores = reading_scores_complete.groupby(['School Name','Grade']).mean()

# Pivot table to put grades as column headers
reading_scores_pivot = reading_scores.pivot_table(index='School Name', columns='Grade', values='Reading Score')
reading_scores_pivot = reading_scores_pivot[['9th','10th','11th','12th']]
reading_scores_pivot

Grade,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [7]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 580, 600, 620, 640, 660]
group_names = ["<$580", "$580-599", "$600-619", "$620-639", "$640-660"]


In [8]:
# New dataframe to hold 'Spending Ranges (Per Student)'
school_budget = pd.DataFrame({
    'Spending Ranges (Per Student)': school_group_type['Per Student Budget'],
    'Average Math Score': school_group_type['Average Math Score'],
    'Average Reading Score': school_group_type['Average Reading Score'],
    '% Passing Math': school_group_type['% Passing Math'],
    '% Passing Reading': school_group_type['% Passing Reading'],
    '% Overall Passing Rate': school_group_type['% Overall Passing Rate']
})

# Bin each row
school_budget['Spending Ranges (Per Student)'] = pd.cut(school_budget['Spending Ranges (Per Student)'], spending_bins, labels=group_names)

# Group by bins
school_budget_ranges = school_budget.groupby(['Spending Ranges (Per Student)']).mean()
school_budget_ranges

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$580,83.274201,83.989488,93.867718,96.539641,95.203679
$580-599,83.476713,83.867873,93.459946,96.439624,94.949785
$600-619,83.839917,84.044699,94.594595,95.945946,95.270270
$620-639,79.474551,82.120471,77.139934,87.468080,82.304007
$640-660,77.023555,80.957446,66.701010,80.675217,73.688113


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [10]:
# New dataframe to hold 'School Size'
school_size = pd.DataFrame({
    'School Size': school_group_type['Total Students'],
    'Average Math Score': school_group_type['Average Math Score'],
    'Average Reading Score': school_group_type['Average Reading Score'],
    '% Passing Math': school_group_type['% Passing Math'],
    '% Passing Reading': school_group_type['% Passing Reading'],
    '% Overall Passing Rate': school_group_type['% Overall Passing Rate']
})

# Bin each row
school_size['School Size'] = pd.cut(school_size['School Size'], size_bins, labels=group_names)

# Group by bins
school_size_ranges = school_size.groupby(['School Size']).mean()
school_size_ranges


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [11]:
# Group schools by 'School Type' 
school_type_scores = school_group_type.groupby('School Type').mean()
school_type_scores = school_type_scores[['Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]
school_type_scores

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


# Include a written description of at least two observable trends based on the data.

1. When comparing the top and bottom performing schools, the difference in Math Scores and Reading Scores becomes stark. In the top schools, the passing rates for Math and Reading are similar, at less than 5% difference between them and both at very high rates. However in the bottom schools, the % Passing Math rates are much lower than the % Passing Reading Rates, around 10%-15% lower with the % Passing Math in the 60s. This shows that the Math Scores have a lot of room for improvement. If we focused on improving the Math Scores of the bottom performing schools, it could greatly improve their overall passing rates. 

2. Interestingly, there seems to be a decline in the scores and passing rates when the school budget surpasses around $620 per student. What could be some possible reasons? Is the extraneous budget being spent on things that are not directly improving student scores, possibly even deterring/distracting from them? We could investigate some of the expenses in the budget that may be correlated with this decline.
